In [ ]:
import warnings
warnings.filterwarnings('ignore')
from urllib.parse import quote_plus
# setting up the environment -> openai api key
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
db_user=os.getenv("db_user")
db_password=quote_plus(os.getenv("db_password"))
db_host=os.getenv("db_host")
db_name=os.getenv("db_name")

In [7]:
# initialize the llm model -> OpenAI
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.)
print(llm("what is 5+5?").strip())

5+5 is 10.


In [8]:
# SQL DB object:
from langchain.utilities import SQLDatabase

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [15]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm=llm,
                                     db=db,
                                     verbose=1)

In [29]:
qns1 = db_chain.run("How many t-shirts do we have left for nike in extra small size and white color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT COUNT(*) AS total FROM t_shirts WHERE brand = 'Nike' AND size = 'XS' AND color = 'White'
SQLResult: [(1,)]
Answer:1
> Finished chain.


In [17]:
qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price * stock_quantity) AS total_price FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('23239'),)]
Answer:The total price of the inventory for all small size t-shirts is $232.39.
> Finished chain.


In [ ]:
qns2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' LIMIT 5
SQLResult: [(Decimal('23239'),)]
Answer:The total value of all t-shirts in size S is $23,239.
> Finished chain.


Thus, the llm did well.

In [19]:
qns3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT SUM(price - (price * (pct_discount/100))) AS revenue FROM t_shirts INNER JOIN discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() = 'today' LIMIT 5;

OperationalError: (pymysql.err.OperationalError) (1525, "Incorrect DATE value: 'today'")
[SQL: SELECT SUM(price - (price * (pct_discount/100))) AS revenue FROM t_shirts INNER JOIN discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() = 'today' LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

Above, it returned a wrong query which generated an error during query execution. It thinks discount table would have start and end date which is normally true but in our table there is no start or end date column. One thing we can do here is run the query directly.

In [20]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:SELECT SUM(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) AS total_revenue FROM
(SELECT SUM(price*stock_quantity) AS total_amount, t_shirt_id FROM t_shirts WHERE brand = 'Levi'
GROUP BY t_shirt_id) a LEFT JOIN discounts ON a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('28003.000000'),)]
Answer:The total revenue for Levi brand t-shirts after applying any discounts is $28,003.
> Finished chain.


Now this is not much interesting because what is the point of giving it the ready made query? Well, we will use this same query later on for few shot learning

In [21]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('28003'),)]
Answer:The total revenue for all Levi t-shirts is $28,003.
> Finished chain.


In [22]:
qns5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT COUNT(*) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(5,)]
Answer:5
> Finished chain.


Once again above is the wrong answer. We need to use SUM(stock_quantity). Let's run the query explicitly. We will use the result of this query later on in the notebook

In [23]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('370'),)]
Answer:370
> Finished chain.


**Few shot learning**
We will use few shot learning to fix issues we have seen so far

In [44]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},

    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},

    {'Question': "If we have to sell all the Levi's T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
    'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
    """,
    'SQLResult': "Result of the SQL query",
    'Answer': qns3} ,

     {'Question' : "If we have to sell all the Levi's T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
      
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     }
]

**Creating Semantic Similarity Based example selector**

* Create embedding on the few_shots
* Store the embeddings in Chroma DB
* Retrieve the the top most Semantically close example from the vector store

In [45]:
to_vectorize = [" ".join(example.values()) for example in few_shots]

In [47]:
to_vectorize

["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 1",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query The total value of all t-shirts in size S is $23,239.",
 "If we have to sell all the Levi's T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n     Result of the SQL query The total revenue for Levi brand t-shirts after applying any discounts is $28,003.",
 "If we have to sell all the Levi's T-shirts today. How much

In [48]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

class HFEmbeddingAdapter(HuggingFaceEmbeddings):
    def __init__(self, model_name: str, **kwargs):
        super().__init__(model_name=model_name, **kwargs)

    def __call__(self, input):
        # Chroma.from_texts will call __call__ with a list of strings
        return self.embed_documents(input)

# now use the adapter in place of HuggingFaceEmbeddings
embeddings = HFEmbeddingAdapter(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = Chroma.from_texts(
    to_vectorize,
    embedding=embeddings,
    metadatas=few_shots
)

In [50]:
from langchain.prompts import SemanticSimilarityExampleSelector
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore, 
    k=2,)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLResult': 'Result of the SQL query',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'Answer': '1'},
 {'Question': 'How much is the total price of the inventory for all S-size t-shirts?',
  'Answer': 'The total value of all t-shirts in size S is $23,239.',
  'SQLResult': 'Result of the SQL query',
  'SQLQuery': "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'"}]

In [51]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [52]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


**Setting up PromptTemplete using input variables**

In [53]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [54]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [55]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [56]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [57]:
new_chain("How many white color Levi's shirt I have?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's shirt I have?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('370'),)]
Answer:370
> Finished chain.


{'query': "How many white color Levi's shirt I have?", 'result': '370'}

Now this is working ok. Previously for this same question it was giving wrong answer because it did not use SUM clause around stock_quantity column

In [58]:
new_chain("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('23239'),)]
Answer:The total value of all t-shirts in size S is $23,239.
> Finished chain.


{'query': 'How much is the price of the inventory for all small size t-shirts?',
 'result': 'The total value of all t-shirts in size S is $23,239.'}

In [59]:
new_chain("How much is the price of all white color levi t shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of all white color levi t shirts?
SQLQuery:SELECT sum(price*stock_quantity) as total_price FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('7538'),)]
Answer:The total price of all white Levi t-shirts is $7,538.
> Finished chain.


{'query': 'How much is the price of all white color levi t shirts?',
 'result': 'The total price of all white Levi t-shirts is $7,538.'}

In [60]:
new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike'group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('21073.900000'),)]
Answer:The total revenue for Nike brand t-shirts after applying any discounts is $21,073.90.
> Finished chain.


{'query': 'If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': 'The total revenue for Nike brand t-shirts after applying any discounts is $21,073.90.'}

In [61]:
new_chain("If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Van Huesen'group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('26008.000000'),)]
Answer:The total revenue for Van Heusen brand t-shirts after applying any discounts is $26,008.
> Finished chain.


{'query': 'If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': 'The total revenue for Van Heusen brand t-shirts after applying any discounts is $26,008.'}

In [62]:
new_chain.run('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')



> Entering new SQLDatabaseChain chain...
How much revenue  our store will generate by selling all Van Heuson TShirts without discount?
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Van Huesen'
SQLResult: [(Decimal('27645'),)]
Answer:The total revenue for all Van Heusen t-shirts is $27,645.
> Finished chain.


'The total revenue for all Van Heusen t-shirts is $27,645.'